# PROBLEM SET 3

### Computational Methods in Economics

### Marcelo Alonso

##### AI USE: Eu utilizei o ChatGPT para me auxiliar na **organização e otimização do código**, no **reconhecimento da sintaxe das funções do Julia** e na **correção ortográfica/concordância** das minhas respostas. 


In [ ]:
using Pkg

# 1) Ativa o ambiente local do projeto usando o diretório onde o script está salvo (relative path)
Pkg.activate(@__DIR__)

# 2) Resolve as dependências do ambiente (confere se as versões dos pacotes são compatíveis)
Pkg.resolve()

# 3) Instancia o ambiente, instalando os pacotes conforme definido nos arquivos Project.toml e Manifest.toml
Pkg.instantiate()

# 4) Define a lista dos pacotes necessários para o projeto
packages = [
    "Plots",
    "Interpolations",
    "DataFrames",
    "DataInterpolations",
    "Random",
    "Distributions",
    "BenchmarkTools",
    "Roots",
    "Optim",
    "NLopt",
    "OptimizationNLopt",
    "JuMP",
    "Ipopt",
    "FastGaussQuadrature",
    "Statistics"
]

# 5) Adiciona os pacotes da lista, instalando-os se ainda não estiverem presentes
Pkg.add(packages)


# QUESTION 1


### a) We are interested in computing $E[X]$ using numerical integration. What is the value of   $E[X]$ using Gauss-Hermite quadrature with $n = 3$ nodes? And with $n = 5$ nodes? And with   $n = 10$ nodes?

### RESPOSTA DO ITEM A:


Queremos calcular numericamente a esperança de uma variável normal padrão:

$ \mathbb{E}[X] = \int_{-\infty}^{\infty} x \cdot \frac{1}{\sqrt{2\pi}} e^{-x^2/2} \, dx $

Como a quadratura de Gauss-Hermite aproxima integrais da forma:

$ \int_{-\infty}^{\infty} f(t) \cdot e^{-t^2} \, dt \approx \sum_{i=1}^{n} w_i f(x_i) $,

fazemos a mudança de variável $x = \sqrt{2}t$, com $dx = \sqrt{2}dt$, para reescrever a esperança como:

$ \mathbb{E}[X] = \frac{\sqrt{2}}{\sqrt{\pi}} \int_{-\infty}^{\infty} t \cdot e^{-t^2} \, dt $

Essa forma é compatível com a quadratura de Gauss-Hermite, que usamos no código com:

$ \mathbb{E}[X] \approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^{n} w_i \cdot (\sqrt{2} x_i) $


### Conforme aumentamos o número de pontos nos aproximamos cada vez mais do 0, de modo que: 

### Usando n = 3 pontos, E[X] ≈ -8.1428914568091665e-16

### Usando n = 5 pontos, E[X] ≈ -6.811841891753822e-16

### Usando n = 10 pontos, E[X] ≈ 7.585025094984501e-18


In [6]:
using FastGaussQuadrature


function gauss_hermite_EX(n)
    
    x, w = gausshermite(n)

    return (1 / sqrt(pi)) * sum(w .* (sqrt(2) .* x))
end

for n in (3, 5, 10)
    valor_aproximado = gauss_hermite_EX(n)
    println("Usando n = $n pontos, E[X] ≈ $valor_aproximado")
end


Usando n = 3 pontos, E[X] ≈ -8.1428914568091665e-16
Usando n = 5 pontos, E[X] ≈ -6.811841891753822e-16
Usando n = 10 pontos, E[X] ≈ 7.585025094984501e-18


### b) What wold be the estimated value of $E[X]$ using Monte Carlo integration with $n = 10^2$   simulations? And with $n = 10^4$ simulations? And with $n = 10^6$ simulations?

### RESPOSTA DO ITEM B:

### O método gera N amostras com média 0 e variância 1 e depois tira a média de todas essas amostras. Assim como no item a, conforme aumentamos o número de simulações nos aproximamos cada vez mais do 0:

### Usando n = 100, estimativa de E[X] ≈ 0.06581665359085116

### Usando n = 10000, estimativa de E[X] ≈ -0.002516924702179371

### Usando n = 1000000, estimativa de E[X] ≈ 0.0012966637350114894

In [10]:
using Random, Statistics


function mc_estimate_mean(N::Int)
    # Gera N amostras de N(0,1)
    samples = randn(N)
    # Retorna a média das amostras
    return mean(samples)
end

# Semente reprodutibilidade
Random.seed!(1234)

# Lista dos tamanhos de amostra
ns = [10^2, 10^4, 10^6]

for n in ns
    estimativa = mc_estimate_mean(n)
    println("Usando n = $n, estimativa de E[X] ≈ $estimativa")
end


Usando n = 100, estimativa de E[X] ≈ 0.06581665359085116
Usando n = 10000, estimativa de E[X] ≈ -0.002516924702179371
Usando n = 1000000, estimativa de E[X] ≈ 0.0012966637350114894


### c) Now we are interested in computing $E[\max(1, X)]$. What is the analytical result? Hint: use   the truncated normal distribution.

### RESPOSTA DO ITEM C:



Para $X \sim \mathcal{N}(0,1)$, note que:

- Se $X \le 1$, então $\max(1, X) = 1$;
- Se $X > 1$, então $\max(1, X) = X$.

Usando a fórmula de esperança condicional:

$ \mathbb{E}[\max(1, X)] = 1 \cdot P(X \le 1) + \mathbb{E}[X \mid X > 1] \cdot P(X > 1) $

Sabemos que:

- $P(X \le 1) = \Phi(1)$  
- $P(X > 1) = 1 - \Phi(1)$  
- $\mathbb{E}[X \mid X > 1] = \dfrac{\phi(1)}{1 - \Phi(1)}$

Substituindo tudo na fórmula:

$ \mathbb{E}[\max(1, X)] = \Phi(1) + (1 - \Phi(1)) \cdot \dfrac{\phi(1)}{1 - \Phi(1)} = \Phi(1) + \phi(1) $

Com valores numéricos:

- $\Phi(1) \approx 0.8413$
- $\phi(1) \approx 0.24197$

Portanto:

$ \mathbb{E}[\max(1, X)] \approx 0.8413 + 0.2420 = 1.0833 $


In [14]:
using Distributions, Statistics, Random



# 1) Usando distribuição normal truncada

function E_max_1_truncdist()
    # Probabilidade de X ≤ 1
    p = cdf(Normal(), 1)          # Φ(1)
    # Definimos a Normal truncada de [1, ∞)
    T = truncated(Normal(), lower=1)
    # E[X | X > 1]
    E_cond = mean(T)
    # Combinação das duas partes
    return p*1 + (1 - p)*E_cond
end


# 2) Via Monte Carlo

function E_max_1_montecarlo(N = 10^6)
    # Gera N amostras de N(0,1)
    x = randn(N)
    # Calcula a média de max(1, x)
    return mean(max(1, xi) for xi in x)
end

# Para reprodutibilidade
Random.seed!(1234)


# 1) Normal truncada
println("Normal truncada = ", E_max_1_truncdist())

# 2) Monte Carlo (N = 10^6)
println("Monte Carlo     = ", E_max_1_montecarlo())


Normal truncada = 1.0833154705876862
Monte Carlo     = 1.0837295253972041


### d) Using numerical methods, how do your answers from a)-b) change when considering the expectation of $\max(1, X)$ instead of $X$?

### RESPOSTA DO ITEM D:



#### Gauss-Hermite Quadrature

A esperança que queremos calcular tem a forma:

$ \mathbb{E}[\max(1, X)] = \int_{-\infty}^{\infty} \max(1, x) \cdot \phi(x) dx $

Com isso, reescrevemos a esperança como:

$ \mathbb{E}[\max(1, X)] = \frac{1}{\sqrt{\pi}} \int_{-\infty}^{\infty} \max(1, \sqrt{2}t) \cdot e^{-t^2} dt $

Agora a integral está no formato apropriado para Gauss-Hermite. Avaliamos a função $f(t) = \max(1, \sqrt{2}t)$ nos nós $t_i$ fornecidos pela quadratura e ponderamos pelos pesos $w_i$, obtendo a aproximação:

$ \mathbb{E}[\max(1, X)] \approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^{n} w_i \cdot \max(1, \sqrt{2}t_i) $

---

#### Monte Carlo

O método de Monte Carlo é mais direto: geramos $N$ amostras de $X \sim \mathcal{N}(0,1)$ e calculamos a média da função $\max(1, X)$ sobre essas amostras:

$ \mathbb{E}[\max(1, X)] \approx \frac{1}{N} \sum_{i=1}^{N} \max(1, x_i) $

À medida que $N$ aumenta, a estimativa converge para o valor teórico graças à Lei dos Grandes Números.

---

####  Resultados

####  Gauss-Hermite Quadrature:

#### n = 3, E[max(1, X)] ≈ 1.1220084679281461

#### n = 5, E[max(1, X)] ≈ 1.0998806870765288

#### n = 10, E[max(1, X)] ≈ 1.0934755830123923

#### Monte Carlo Estimation:

#### N = 100, E[max(1, X)] ≈ 1.1155418134953896

#### N = 10000, E[max(1, X)] ≈ 1.082764603825027

#### N = 1000000, E[max(1, X)] ≈ 1.0834872351084592



In [ ]:
using FastGaussQuadrature, Statistics, Random, Distributions


# 1) Gauss-Hermite Quadrature para E[max(1, X)]

function gauss_hermite_Emax1(n)
    # Nós (x) e pesos (w) para ∫ f(t) e^(-t^2) dt
    x, w = gausshermite(n)
    
    # Avaliamos max(1, √2 * x_i) em cada nó e multiplicamos pelos pesos
    valores = [max(1, sqrt(2)*xi) for xi in x]
    
    
    return (1 / sqrt(pi)) * sum(w .* valores)
end


# 2) Monte Carlo para E[max(1, X)]

function mc_Emax1(N)
    # Gera N amostras de N(0,1)
    samples = randn(N)
    # Calcula a média de max(1, x_i)
    return mean(max(1, x) for x in samples)
end

# Semente para reprodutibilidade
Random.seed!(1234)

println("=== Gauss-Hermite Quadrature ===")
for n in (3, 5, 10)
    val = gauss_hermite_Emax1(n)
    println("n = $n, E[max(1, X)] ≈ $val")
end

println("\n=== Monte Carlo Estimation ===")
for N in (10^2, 10^4, 10^6)
    val = mc_Emax1(N)
    println("N = $N, E[max(1, X)] ≈ $val")
end



=== Gauss-Hermite Quadrature ===
n = 3, E[max(1, X)] ≈ 1.1220084679281461
n = 5, E[max(1, X)] ≈ 1.0998806870765288
n = 10, E[max(1, X)] ≈ 1.0934755830123923

=== Monte Carlo Estimation ===
N = 100, E[max(1, X)] ≈ 1.1155418134953896
N = 10000, E[max(1, X)] ≈ 1.082764603825027
N = 1000000, E[max(1, X)] ≈ 1.0834872351084592


# QUESTION 2

### 1. Using Gauss-Hermite quadrature, compute the expected value of $u$. 

### RESPOSTA DO ITEM 2.1:

A forma mais direta de usar Gauss–Hermite em duas dimensões para E[max(X,Y)] (onde X e Y são N(0,1) independentes) é aproveitar que a densidade conjunta é $ \frac{1}{2\pi} \exp\left( -\frac{x^2 + y^2}{2} \right) $, e então aproximar a integral bidimensional via quadratura de Gauss–Hermite. 


A esperança que queremos é:

$ E[\max(X,Y)] = \int_{-\infty}^\infty \int_{-\infty}^\infty \max(x,y) \cdot \frac{1}{2\pi} \exp\left( -\frac{x^2 + y^2}{2} \right) dx\,dy. $


No problema 2D, a mudança de variável resulta em $x = \sqrt{2}\,u$ e $y = \sqrt{2}\,v$. Com essa mudança, temos:

$ dx = \sqrt{2}du \quad \Rightarrow \quad dy = \sqrt{2}dv $

$ \exp\left(-\frac{x^2 + y^2}{2} \right) = \exp\left( -\frac{2u^2 + 2v^2}{2} \right) = \exp(-u^2 - v^2) $


Substituindo essas expressões na integral original, obtemos:


$ \frac{1}{\pi} \int_{-\infty}^\infty \int_{-\infty}^\infty \max(\sqrt{2}u, \sqrt{2}v) \cdot e^{-u^2} e^{-v^2} du dv. $


O resultado final é que, para aproximar $ \iint \frac{1}{2\pi} \exp\left(-\frac{x^2 + y^2}{2} \right) f(x,y)\,dx\,dy, $ podemos usar:

$ \frac{1}{\pi} \sum_{i=1}^n \sum_{j=1}^n w_i w_j f\left( \sqrt{2}x_i, \sqrt{2}x_j \right). $

### No nosso caso, $f(x,y) = \max(x,y)$ e encontramos 0.5640735841549794.















In [24]:
using FastGaussQuadrature


function expected_max_gauss_hermite(n::Int)
    # Nós e pesos para a quadratura Gauss–Hermite unidimensional
    x, w = gausshermite(n)

    # Soma bidimensional
    s = 0.0
    for i in 1:n
        for j in 1:n
            # max(sqrt(2)*x[i], sqrt(2)*x[j]) é a função f(x,y)
            s += w[i] * w[j] * max( sqrt(2)*x[i], sqrt(2)*x[j] )
        end
    end

   
    return s / π
end

# Calcula a aproximação de E[max(X,Y)] com n = 20
println("Ordem da quadratura = 2000")
val = expected_max_gauss_hermite(2000)
println("Aproximação de E[max(X,Y)] = $val")

# Comparando com o valor analítico 1/sqrt(pi)
println("Valor exato = ", 1/sqrt(π))
println("Erro absoluto ≈ ", abs(val - 1/sqrt(π)))


Ordem da quadratura = 2000
Aproximação de E[max(X,Y)] = 0.5640735841549794
Valor exato = 0.5641895835477563
Erro absoluto ≈ 0.00011599939277684435


### 2. Using Monte Carlo integration, compute the expected value of $u$.

### RESPOSTA DO ITEM 2.2:

### Da mesma forma, o método de Monte Carlo aproxima o valor esperado conforme aumentamos o número de simulações e chegamos em:

#### Estimativa Monte Carlo de E[max(X,Y)] com N = 1000000: 0.562641391343698

In [26]:
using Random, Statistics, Distributions


function monte_carlo_max_expectation(N::Int)
    # Definindo a distribuição normal padrão
    dist = Normal(0, 1)

    # Gerando N amostras independentes de X e Y
    X = rand(dist, N)
    Y = rand(dist, N)

    # Calculando max(x, y) para cada par (x, y)
    max_vals = max.(X, Y)

    # Retornando a média das máximas — estimativa de E[max(X,Y)]
    return mean(max_vals)
end


N = 10^6  # número de simulações
estimativa = monte_carlo_max_expectation(N)
println("Estimativa Monte Carlo de E[max(X,Y)] com N = $N: $estimativa")

# Valor analítico 
println("Valor exato: ", 1 / sqrt(π))


Estimativa Monte Carlo de E[max(X,Y)] com N = 1000000: 0.562641391343698
Valor exato: 0.5641895835477563


# QUESTION 3



In [ ]:

# Função para aplicar a Regra do Trapézio
# f: função a ser integrada
# a, b: limites de integração
# n: número de subintervalos (ou seja, usaremos n+1 pontos)
# ---------------------------------------------------------
function trapezio(f, a, b, n)
    h = (b - a) / n                # tamanho de cada subintervalo
    x = range(a, b, length = n+1)  # cria n+1 pontos igualmente espaçados
    # Soma dos termos extremos
    soma = f(x[1]) + f(x[end])
    # Soma dos termos internos (cada um é multiplicado por 2 na fórmula)
    for k in 2:n
        soma += 2 * f(x[k])
    end
    # Multiplica por h/2 para obter a aproximação final
    return (h/2) * soma
end


# Valores de n solicitados
ns = [3, 5, 10, 15, 20]



### a) $\int_0^1 x \, dx$  

### RESPOSTA DO ITEM A:


In [31]:
# a) ∫₀¹ x dx = 1/2
valor_exato_a = 0.5

f_a(x) = x


println("Aproximação usando a Regra do Trapézio:\n")
for n in ns
    # Calcula as aproximações
    I_a = trapezio(f_a, 0, 1, n)
   
    
    println("--------------------------------------------------")
    println(" n = $n")
    println(" Integral de x                : $I_a    (Erro = $(abs(I_a - valor_exato_a)))")
end

println("\nValor exato:")
println("∫₀¹ x dx               = $valor_exato_a")




Aproximação usando a Regra do Trapézio:

--------------------------------------------------
 n = 3
 Integral de x                : 0.5    (Erro = 0.0)
--------------------------------------------------
 n = 5
 Integral de x                : 0.5    (Erro = 0.0)
--------------------------------------------------
 n = 10
 Integral de x                : 0.5    (Erro = 0.0)
--------------------------------------------------
 n = 15
 Integral de x                : 0.5    (Erro = 0.0)
--------------------------------------------------
 n = 20
 Integral de x                : 0.5    (Erro = 0.0)

Valor exato:
∫₀¹ x dx               = 0.5


### b) $\int_0^1 x \sin(x) \, dx$  

### RESPOSTA DO ITEM B:


In [32]:
# b) ∫₀¹ x sin(x) dx = -x cos(x) + sin(x) avaliado em 0 até 1 = sin(1) - cos(1)
valor_exato_b = sin(1) - cos(1)


f_b(x) = x * sin(x)


println("Aproximação usando a Regra do Trapézio:\n")
for n in ns
    # Calcula a aproximação
    I_b = trapezio(f_b, 0, 1, n)
    
    
    println("--------------------------------------------------")
    println(" n = $n")
    println(" Integral de x*sin(x)         : $I_b    (Erro = $(abs(I_b - valor_exato_b)))")
end

println("\nValor exato:")
println("∫₀¹ x*sin(x) dx        = $valor_exato_b")



Aproximação usando a Regra do Trapézio:

--------------------------------------------------
 n = 3
 Integral de x*sin(x)         : 0.31401564223860784    (Erro = 0.0128469632988511)
--------------------------------------------------
 n = 5
 Integral de x*sin(x)         : 0.30578141044861207    (Erro = 0.004612731508855328)
--------------------------------------------------
 n = 10
 Integral de x*sin(x)         : 0.30232058249393656    (Erro = 0.0011519035541798228)
--------------------------------------------------
 n = 15
 Integral de x*sin(x)         : 0.3016805309189573    (Erro = 0.0005118519792005616)
--------------------------------------------------
 n = 20
 Integral de x*sin(x)         : 0.30145657498119877    (Erro = 0.0002878960414420262)

Valor exato:
∫₀¹ x*sin(x) dx        = 0.30116867893975674


### c) $\int_0^1 \sqrt{1 - x^2} \, dx$

### RESPOSTA DO ITEM C:

In [33]:
# c) ∫₀¹ sqrt(1 - x^2) dx = π/4 (área de 1/4 de círculo de raio 1)
valor_exato_c = pi/4

f_c(x) = sqrt(1 - x^2)

println("Aproximação usando a Regra do Trapézio:\n")
for n in ns
    # Calcula a aproximação
    I_c = trapezio(f_c, 0, 1, n)
    
    println("--------------------------------------------------")
    println(" n = $n")
    println(" Integral de sqrt(1 - x^2)     : $I_c    (Erro = $(abs(I_c - valor_exato_c)))")
end

println("\nValor exato:")
println("∫₀¹ sqrt(1 - x^2) dx   = $valor_exato_c")


Aproximação usando a Regra do Trapézio:

--------------------------------------------------
 n = 3
 Integral de sqrt(1 - x^2)     : 0.7293883446939977    (Erro = 0.056009818703450565)
--------------------------------------------------
 n = 5
 Integral de sqrt(1 - x^2)     : 0.7592622072208878    (Erro = 0.026135956176560504)
--------------------------------------------------
 n = 10
 Integral de sqrt(1 - x^2)     : 0.7761295815620796    (Erro = 0.009268581835368717)
--------------------------------------------------
 n = 15
 Integral de sqrt(1 - x^2)     : 0.7803478531188924    (Erro = 0.005050310278555847)
--------------------------------------------------
 n = 20
 Integral de sqrt(1 - x^2)     : 0.7821162199387454    (Erro = 0.0032819434587029184)

Valor exato:
∫₀¹ sqrt(1 - x^2) dx   = 0.7853981633974483


# QUESTION 4



In [ ]:
###############################################################################
# Bloco inicial: Definições gerais para evitar repetição
###############################################################################

# Funções de derivada numérica (Diferenças Finitas Centrais)
deriv_central_2pts(f, x, h) = (f(x+h) - f(x-h)) / (2h)
deriv_central_4pts(f, x, h) = (-f(x+2h) + 8*f(x+h) - 8*f(x-h) + f(x-2h)) / (12*h)

# Funções f(x) para cada item
fa(x) = x^2
fb(x) = log(x)
fc(x) = x*sin(x)

# Derivadas analíticas exatas (nomes corrigidos!)
dfa(x) = 2x
dfb(x) = 1/x
dfc(x) = sin(x) + x*cos(x)

# Pontos e passos de h
x_a = 5
x_b = 10
x_c = 1
hs  = [0.001, 0.005, 0.01, 0.05]

### a) $f(x) = x^2$ at $x = 5$  

### RESPOSTA DO ITEM A:

In [44]:

println("------------------------------------------------------------")
println("Item (a): f(x) = x^2 em x = 5")
println("Derivada exata em x=5: f'(5) = 10\n")
for h in hs
    approx_2 = deriv_central_2pts(fa, x_a, h)
    approx_4 = deriv_central_4pts(fa, x_a, h)
    err_2 = abs(approx_2 - dfa(x_a))
    err_4 = abs(approx_4 - dfa(x_a))
    println("  h = $h")
    println("    2 pts: $approx_2   | erro = $err_2")
    println("    4 pts: $approx_4   | erro = $err_4\n")
end

------------------------------------------------------------
Item (a): f(x) = x^2 em x = 5
Derivada exata em x=5: f'(5) = 10

  h = 0.001
    2 pts: 10.00000000000334   | erro = 3.339550858072471e-12
    4 pts: 10.000000000004524   | erro = 4.524380869952438e-12

  h = 0.005
    2 pts: 9.999999999999787   | erro = 2.1316282072803006e-13
    4 pts: 9.999999999999787   | erro = 2.1316282072803006e-13

  h = 0.01
    2 pts: 9.999999999999787   | erro = 2.1316282072803006e-13
    4 pts: 9.999999999999728   | erro = 2.717825964282383e-13

  h = 0.05
    2 pts: 9.999999999999964   | erro = 3.552713678800501e-14
    4 pts: 9.999999999999975   | erro = 2.4868995751603507e-14



### b) $f(x) = \log(x)$ at $x = 10$  

### RESPOSTA DO ITEM B:

In [38]:

println("------------------------------------------------------------")
println("Item (b): f(x) = log(x) em x = 10")
println("Derivada exata em x=10: f'(10) = 0.1\n")
for h in hs
    approx_2 = deriv_central_2pts(fb, x_b, h)
    approx_4 = deriv_central_4pts(fb, x_b, h)
    err_2 = abs(approx_2 - dfb(x_b))
    err_4 = abs(approx_4 - dfb(x_b))
    println("  h = $h")
    println("    2 pts: $approx_2   | erro = $err_2")
    println("    4 pts: $approx_4   | erro = $err_4\n")
end

------------------------------------------------------------
Item (b): f(x) = log(x) em x = 10
Derivada exata em x=10: f'(10) = 0.1

  h = 0.001
    2 pts: 0.10000000033327794   | erro = 3.332779330289526e-10
    4 pts: 0.09999999999984095   | erro = 1.590533260653615e-13

  h = 0.005
    2 pts: 0.10000000833332301   | erro = 8.333323003872906e-9
    4 pts: 0.09999999999995939   | erro = 4.0620284913472915e-14

  h = 0.01
    2 pts: 0.10000003333334728   | erro = 3.333334727684267e-8
    4 pts: 0.09999999999992608   | erro = 7.392697565222761e-14

  h = 0.05
    2 pts: 0.10000083334583465   | erro = 8.33345834644339e-7
    4 pts: 0.09999999995000007   | erro = 4.999993474807951e-11



### c) $f(x) = x * \sin(x)$ at $x = 1$  

### RESPOSTA DO ITEM C:

In [41]:

println("------------------------------------------------------------")
println("Item (c): f(x) = x*sin(x) em x = 1")
println("Derivada exata em x=1: f'(1) = sin(1) + cos(1)\n")
for h in hs
    approx_2 = deriv_central_2pts(fc, x_c, h)
    approx_4 = deriv_central_4pts(fc, x_c, h)
    err_2 = abs(approx_2 - dfc(x_c))
    err_4 = abs(approx_4 - dfc(x_c))
    println("  h = $h")
    println("    2 pts: $approx_2   | erro = $err_2")
    println("    4 pts: $approx_4   | erro = $err_4\n")
end

------------------------------------------------------------
Item (c): f(x) = x*sin(x) em x = 1
Derivada exata em x=1: f'(1) = sin(1) + cos(1)

  h = 0.001
    2 pts: 1.3817727798901003   | erro = 5.107859359920752e-7
    4 pts: 1.3817732906757085   | erro = 3.277378368693462e-13

  h = 0.005
    2 pts: 1.3817605210538209   | erro = 1.276962221541389e-5
    4 pts: 1.3817732905770894   | erro = 9.894685071287768e-11

  h = 0.01
    2 pts: 1.381722212483999   | erro = 5.1078192037312675e-5
    4 pts: 1.3817732890935066   | erro = 1.5825296628690921e-9

  h = 0.05
    2 pts: 1.380496573238128   | erro = 0.001276717437908248
    4 pts: 1.3817723019794308   | erro = 9.8869660547507e-7



### d) How do your results compare with the analytical solution?

### RESPOSTA DO ITEM D:


Ao aproximar derivadas por diferenças finitas, o **erro total** resulta da soma de duas fontes conflitantes:


- **$h$ pequeno**: melhora a aproximação teórica (reduz truncamento), mas **piora a estabilidade numérica** (erro de arredondamento).
- **$h$ grande**: reduz o cancelamento numérico, mas aumenta o erro de truncamento.

#### **Item (a): $f(x) = x^2$ em $x = 5$**
- Os erros são extremamente baixos mesmo com $h = 0.05$, tanto para 2 quanto para 4 pontos.
- Isso confirma que, para funções polinomiais simples, as fórmulas de diferença central **já são exatas**, e o erro numérico é dominado pelo arredondamento.

#### **Item (b): $f(x) = \log(x)$ em $x = 10$**
- Aqui a natureza da função traz mais sensibilidade a erros de arredondamento, principalmente para $h$ muito pequeno.
- O **erro de 4 pontos com $h = 0.005$ foi menor que com $h = 0.001$**, revelando o ponto ótimo entre truncamento e cancelamento.
- Isso ilustra bem a observação de que **usar um $h$ *pequeno demais* pode ser pior** que um valor moderado como 0.005.

#### **Item (c): $f(x) = x\sin(x)$ em $x = 1$**

- Para 4 pontos, o erro se manteve extremamente baixo mesmo para $h$ maiores, indicando robustez.
- Para 2 pontos, o erro cresceu com $h$, como esperado — pois o truncamento domina.
